## Drug Induced Effects on Interferon Signalling

In this case study, we focus on a Boolean network which models a complex human cell signalling pathway, describing the activity of type-1 interferons and related molecules interacting with the SARS-COV-2 virion. The network is designed to explain the dynamics of *innate immune response* (**IIR**) -- an important biological phenotype corresponding to the activity of the immune system, and *inflammation* (**INFL**) -- a biological phenotype causing an overreaction that might cause severe health problems in affected tissues (e.g., cytokine storm \[1]).

We have adopted the BN model of interferon type-1 pathway produced by the COVID-19 Disease Map project \[2]. The original BN model has been constructed from the openly accessible [SBGN curated pathway](https://fairdomhub.org/models/713) by employing the automated inference algorithm CaSQ \[3] (see `interferon-pathway.sbml` available online under revision [51a2374f](https://git-r3lab.uni.lu/covid/models/-/blob/master/Executable%20Modules/SBML_qual_build/sbml/Interferon1_stable.sbml)). CaSQ is producing ready-to-simulate (non-parametrised) BNs. Logical parameters here represent input variables.

The original BN model contains 68 variables, 190 regulations, and 53 logical parameters. The parameters can be classified into three groups: virus proteins, input signals, and drugs (`Azithromycin_drug`, `GRL0617_drug`, and `MNS_drug`). The phenotypes expressed by the actual cell are represented through special output variables: `ISG_expression_antiviral_response_phenotype` (or **IIR**), `Proinflammatory_cytokine_expression_Inflammation_phenotype` (or **INFL**), and `type_I_IFN_response_phenotype` (or **IFN**).


### Parameter induced multi-stability

Due to the large parameter space, computing the network attractors can take several minutes, but is nevertheless feasible using common hardware:

In [1]:
from pandas import DataFrame
from biodivine_aeon import *

Detected IPython (`ZMQInteractiveShell`). Log level set to `LOG_ESSENTIAL`.


In [2]:
model = BooleanNetwork.from_file('interferon-pathway.sbml')
model = model.inline_inputs() # Turn input variables into explicit parameters. Improves performance.
graph = AsynchronousGraph(model)
print(model)

BooleanNetwork(variables=68, regulations=97, explicit_parameters=53, implicit_parameters=0)


In [3]:
attractors = Attractors.attractors(graph)

Start interleaved transition guided reduction with 2658455991569832000000000000000000000[nodes:2] candidates.
 > Discarded 1329227995784916000000000000000000000 instances based on the BnVariable(24) extended component.
 > State space reduced to 1329227995784916000000000000000000000[nodes:5].
 > Discarded 664613997892458000000000000000000000 instances based on the BnVariable(54) extended component.
 > State space reduced to 664613997892458000000000000000000000[nodes:9].
 > Discarded 332306998946229000000000000000000000 instances based on the BnVariable(31) extended component.
 > State space reduced to 332306998946229000000000000000000000[nodes:12].
 > Discarded 166153499473114500000000000000000000 instances based on the BnVariable(5) extended component.
 > State space reduced to 166153499473114500000000000000000000[nodes:16].
 > Discarded 83076749736557240000000000000000000 instances based on the BnVariable(61) extended component.
 > State space reduced to 830767497365572400000000000000

The automatically generated decision tree for these attractor sets is actually quite manageable (only roughly `50` decision nodes), but it is very deep and thus does not fit onto a single page. A composited screenshot of the original full decision tree is available in `full-tree.png`. A compact representation of the tree is also drawn in the `full-tree.pdf` file.

<p align="center">
    <img src="tree-full.png">
</p>

#### Phenotypes are not correlated with multi-stability

Interestingly, from the logical parameters representing drugs, only the `Azithromycin_drug` was chosen by the automatic tree generation algorithm as relevant. Furthermore, if we perform stability analysis on the individual tree nodes, we can notice that the expression of phenotypes is not tied to the number (or types) of attractors, as we for example see in the T-cell survival case study. 

In particular, the decision tree in `tree-full.pdf` shows the ratio of parametrisations exhibiting the **INFL** phenotype in each leaf node. We see that initially, most decisions do not impact the ratio of parametrisations expressing this phenotype (all nodes stay at `25%`). Only once we condition on `E`, we see some variability. However, even in this case, we do not see a clear "appearance" or "disappearance" of an attractor corresponding to an inflammation phenotype. 

Similarly, for **IIR** there is also no clear relationship between mutli-stability, and the presence of immune response.

Finally, the **IFN** phenotype is not expressed in any attractor of the network for any parametrisation (see below). This is unexpected as it is one of the primary outputs of this network. *At this point, we attribute this to a model error and disregard this phenotype in further analysis.*

In [4]:
# Note that this number does not necessarily mean there are 4 unique attractors
# in some parametrisation. The attractor detection algorithm creates the
# sets opportunistically and does not guarantee that the result is in any sense
# minimal. It is only guaranteed to be correct in the sense that each
# returned set contains exactly one attractor for every included color.
print("Attractor set count:", len(attractors))

attrStates = graph.mk_empty_colored_vertices();
for attr in attractors:
    attrStates = attrStates.union(attr)

print("Unique attr. states:", int(attrStates.vertices().cardinality()))

IIR = graph.mk_subspace({"ISG_expression_antiviral_response_phenotype": True})
INFL = graph.mk_subspace({"Proinflammatory_cytokine_expression_Inflammation_phenotype": True})
IFN = graph.mk_subspace({"type_I_IFN_response_phenotype": True})

attrIIR = attrStates.intersect(IIR)
attrINFL = attrStates.intersect(INFL)
attrIFN = attrStates.intersect(IFN)

print("IIR phenotype colors:", int(attrIIR.colors().cardinality()))
print("INFL phenotype colors:", int(attrINFL.colors().cardinality()))
print("IFN phenotype colors:", int(attrIFN.colors().cardinality()))

Attractor set count: 4
Unique attr. states: 59000
IIR phenotype colors: 6756498952683520
INFL phenotype colors: 2251947990056960
IFN phenotype colors: 0


### Logical parameters driving network phenotypes

To better understand the impact of individual parameters on the network phenotypes, we can manually compute the distribution of relevant parametrisations for each phenotype. In the following, we measure how much a network parameter impacts the expression of a particular phenotype. The number describes a change in the number of parametrisations which express the phenotype with the parameter absent or present. That is, a change of `+75%` for parameter `X` means that fixing `X = true` increases the ratio of parameters expressing a particular phenotype by `75%` compared to cases where `X = false` (e.g. `10%` when `X = false` and `85%` when `X = true`).

Note that due to the high number of parameters, this computation can also take several minutes.

In [5]:
def phenotype_ratio(attr, phenotype):
    """The ratio of the phenotype colors compared to all colors in the attr set."""
    return attr.intersect(phenotype).colors().cardinality() / attr.colors().cardinality()

def check_impact(attr, param, phenotype):
    """Compute the impact of the set of param colors on the expression of phenotype in the attr set."""
    attr_param_on = attr.intersect_colors(param)
    attr_param_off = attr.minus_colors(param)
    ratio_on = phenotype_ratio(attr_param_on, phenotype)
    ratio_off = phenotype_ratio(attr_param_off, phenotype)
    return (ratio_on - ratio_off) * 100.0

table = []
# Note that the model inputs are now represented as parameters, since we transformed
# the model using the `inline_inputs` function.
for i in model.explicit_parameters():
    i_true = graph.mk_function_colors(i, "true")
    i_name = model.get_explicit_parameter_name(i)
    iir_impact = check_impact(attrStates, i_true, IIR)
    infl_impact = check_impact(attrStates, i_true, INFL)
    print(f"Impact of {i_name}: {iir_impact} (IIR) {infl_impact} (INFL)")
    if iir_impact != 0 or infl_impact != 0:
        table.append([i_name, iir_impact, infl_impact])

Impact of NEMO: 0.0 (IIR) 0.0032901763916015625 (INFL)
Impact of NFKB1: 0.0 (IIR) 0.0032901763916015625 (INFL)
Impact of TAK1: 0.0 (IIR) 0.0032901763916015625 (INFL)
Impact of M: -0.0244140625 (IIR) 0.0 (INFL)
Impact of Orf7b: -0.0244140625 (IIR) 0.0 (INFL)
Impact of TRIM25: 0.0 (IIR) 0.0 (INFL)
Impact of pp1ab: 0.0 (IIR) 0.0 (INFL)
Impact of Nsp1: -0.0244140625 (IIR) 0.0 (INFL)
Impact of Nsp10: 0.0 (IIR) 0.0 (INFL)
Impact of N: 0.0 (IIR) 0.0 (INFL)
Impact of TLR9: 0.0 (IIR) 0.0001430511474609375 (INFL)
Impact of Nsp14: 0.0 (IIR) 0.0 (INFL)
Impact of sa3_IFNB1_extracellular_space: 0.0 (IIR) 0.0 (INFL)
Impact of MYD88: 0.0 (IIR) 0.0032901763916015625 (INFL)
Impact of IkB: 0.0 (IIR) 0.0032901763916015625 (INFL)
Impact of JAK1: 0.0 (IIR) 0.0 (INFL)
Impact of RIP1: 0.0 (IIR) 0.0032901763916015625 (INFL)
Impact of Nsp3: -49.9755859375 (IIR) -0.0032901763916015625 (INFL)
Impact of MAPK8: 0.0 (IIR) 0.0 (INFL)
Impact of Viral_dsRNA_rna_endosome: 0.0 (IIR) 0.0012874603271484375 (INFL)
Impact of

In [6]:
DataFrame(table, columns=["Variable", "IIR gain", "INFL gain"])

,Variable,IIR gain,INFL gain
0,NEMO,0.000000,0.003290
1,NFKB1,0.000000,0.003290
2,TAK1,0.000000,0.003290
3,M,-0.024414,0.000000
4,Orf7b,-0.024414,0.000000
5,Nsp1,-0.024414,0.000000
6,TLR9,0.000000,0.000143
7,MYD88,0.000000,0.003290
8,IkB,0.000000,0.003290
9,RIP1,0.000000,0.003290


In this table, we can see that only 32 out of the 53 network parameters actually influence phenotype expression (the ones with no impact are skipped). Furthermore, by the change in expression ratios, we can see which parameter has the highest impact on the network's phenotypes. For example, notice that `Azithromycin_drug` has a high impact on the **IIR** phenotype, while `E` and `MNS_drug` have high impact on **INFL**. Interestingly, `GRL0617_drug` does not appear to influence phenotype expression in any way.

Subsequently, we perform three more experiments, each following different conditions. In the first experiment, we test whether the parameter `Viral_replication_phenotype` has any influence on the impact of other parameters. Parameter `Viral_replication_phenotype` is a major factor in introducing multi-stability (as shown by the decision tree above). We thus want to confirm our hypothesis that the multi-stability caused by this parameter has indeed no effect on network phenotypes.

Additionally, we also test cases where `Azithromycin_drug` is fixed to `true`, and where both `MNS_drug` and `Azithromycin_drug` are fixed to `true`. This should give us a better insight into the influence of these drugs.

In [7]:
virus_detected = graph.mk_function_colors("Viral_replication_phenotype", "true")
states_virus_on = attrStates.intersect_colors(virus_detected)

table2 = []
for i in model.explicit_parameters():
    i_name = model.get_explicit_parameter_name(i)
    i_true = graph.mk_function_colors(i, "true")
    if i_name == 'Viral_replication_phenotype':
        continue
    iir_impact = check_impact(states_virus_on, i_true, IIR)
    infl_impact = check_impact(states_virus_on, i_true, INFL)
    if iir_impact != 0 or infl_impact != 0:
        table2.append([i_name, iir_impact, infl_impact])

DataFrame(table2, columns=["Variable", "IIR gain", "INFL gain"])

,Variable,IIR gain,INFL gain
0,Orf3a,-0.024414,0.000000
1,Orf9b,0.000000,-0.003290
2,RIP1,0.000000,0.003290
3,TAB2,0.000000,0.003290
4,MNS_drug,0.000000,-49.998903
5,Nsp3,-49.975586,-0.003290
6,Orf7b,-0.024414,0.000000
7,IKKa,0.000000,0.003290
8,Orf7a,-0.024414,0.000000
9,Nsp6,-0.024414,0.000000


In [8]:
drug_on = graph.mk_function_colors("Azithromycin_drug", "true")
states_drug_on = attrStates.intersect_colors(drug_on)

table3 = []
for i in model.explicit_parameters():
    i_name = model.get_explicit_parameter_name(i)
    i_true = graph.mk_function_colors(i, "true")
    if i_name == 'Azithromycin_drug':
        continue
    iir_impact = check_impact(states_drug_on, i_true, IIR)
    infl_impact = check_impact(states_drug_on, i_true, INFL)
    if iir_impact != 0 or infl_impact != 0:
        table3.append([i_name, iir_impact, infl_impact])

DataFrame(table3, columns=["Variable", "IIR gain", "INFL gain"])

,Variable,IIR gain,INFL gain
0,E,0.0,49.998569
1,IKKa,0.0,0.004292
2,NFKB1,0.0,0.004292
3,TAB2,0.0,0.004292
4,RELA,0.0,0.004292
5,TAK1,0.0,0.004292
6,TAB1,0.0,0.004292
7,IKKb,0.0,0.004292
8,Orf9b,0.0,-0.004292
9,MNS_drug,0.0,-49.998569


In [9]:
drug_on = graph.mk_function_colors("MNS_drug", "true")
states_drug_on = states_drug_on.intersect_colors(drug_on)

table4 = []
for i in model.explicit_parameters():
    i_name = model.get_explicit_parameter_name(i)
    i_true = graph.mk_function_colors(i, "true")
    if i_name == 'Azithromycin_drug':
        continue
    if i_name == 'Viral_replication_phenotype':
        continue
    if i_name == 'MNS_drug':
        continue
    iir_impact = check_impact(states_drug_on, i_true, IIR)
    infl_impact = check_impact(states_drug_on, i_true, INFL)
    if iir_impact != 0 or infl_impact != 0:
        table4.append([i_name, iir_impact, infl_impact])

DataFrame(table4, columns=["Variable", "IIR gain", "INFL gain"])

,Variable,IIR gain,INFL gain
0,MYD88,0.0,0.005722
1,TLR7,0.0,0.000381
2,RIP1,0.0,0.005722
3,TLR9,0.0,0.000381
4,IKKb,0.0,0.005722
5,TAK1,0.0,0.005722
6,NEMO,0.0,0.005722
7,TREML4,0.0,0.000381
8,TAB1,0.0,0.005722
9,TRIF,0.0,0.005722


In the results, we see that `Viral_replication_phenotype` has indeed no effect on the impact of network parameters on its phenotypes (i.e. the numbers are exactly the same). Meanwhile, we see that after applying `Azithromycin_drug`, **IIR** is always active (and thus parameters do not have impact on it any more). Meanwhile, by applying both drugs, the effect of `E` is eliminated as a factor in inflammation. Also note that the impact of other parameters *grows* once we fix either of these two drugs, as opposed to the case of `Viral_replication_phenotype`, where it stayed the same.

### Full logical phenotype characterisation

Overall, the previous experiments give us a better overview of which parameters are important for phenotype expression and what is the intensity of their influence. We can now use this information to construct decision trees that actually describe when a certain phenotype is expressed. These trees are constructed in AEON by hand, such that we confirm using stability analysis that each unexpanded node only admits a particular phenotype. The tree screenshots are located in `tree-immune-response.png` and `tree-inflammation.png`, with `tree-immune-response.pdf` and `tree-inflammation.pdf` showing a compacted, more readable (but logically equivalent) version of these trees.

Notice that in both cases, the trees contain exactly the parameters identified in our previous experiments as the ones having impact on the corresponding phenotype. These parameters also appear in an order which follows their observed impact. Also note that while the inflammation tree appears to be much more complex than the immune response tree, the inflammation tree actually contains two equivalent branches, and depends on more parameter values (22 vs. 13).

### Discussion

During our analysis of the type-1 interferon pathway, we have discovered several unexpected properties which suggest that the model is incomplete or biologically unsound. In particular, we have shown that the model as is cannot express any activity of interferons (**INF** phenotype) -- one of its main functionalities. Additionally, we showed that replication of the virus inside the cell (i.e. `Viral_replication_phenotype`) does not in any way influence the expression of the two remaining phenotypes. 

We also constructed a bifurcation decision tree characterising the presence of different multi-stable behaviour classes. However, in this tree, we do not observe any meaningful correlation between the number of attractors and the expressed network phenotypes.

Nevertheless, we were able to identify `13` and `22` logical parameters which actually influence the immune response (**IIR**) and inflammation (**INFL**) phenotypes respectively. Three of these (`TREML4`, `Nsp3`, and `Azithromycin_drug`) influence both phenotypes. Based on these parameters, we were able to construct decision trees which fully characterise the expression of phenotypes by this network.

In the tree describing the **IIR** phenotype, we can observe that `Azithromycin_drug` always activates immune response. Meanwhile, in its absence, `Nsp3` (a prominent component of most coronaviruses) enables conditions in which immune response is inactive. Subsequently, other virus proteins (`Nsp*` and `Orf*`) further inhibit the immune response.

Similarly, for the inflammation phenotype, we see `MNS_drug` playing a crucial role by counteracting the influence of the envelope protein `E` as a cause of inflammation. Interestingly, in this case, `Nsp3` actually acts as an inhibitor, with `Viral_dsRNA_rna_endosome` playing a crucial role in the activation of inflammation in the absence of `Nsp3`.

Finally, we can conclude that in this network, `GRL0617_drug` does not have any meaningful impact on either the emergence of multi-stability, or the expression of network phenotypes.

### References

\[1] Hojyo, Shintaro, et al. "How COVID-19 induces cytokine storm with high mortality." Inflammation and regeneration 40.1 (2020): 1-7.

\[2] Ostaszewski, Marek, et al. "COVID-19 Disease Map, building a computational repository of SARS-CoV-2 virus-host interaction mechanisms." Scientific data 7.1 (2020): 1-4.

\[3] Aghamiri, Sara Sadat, et al. "Automated inference of Boolean models from molecular interaction maps using CaSQ." Bioinformatics 36.16 (2020): 4473-4482.